In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import os

# 0. Prepare

In [2]:
# setup spark session 
sc = SparkContext()
spark = SparkSession(sc)

22/05/19 11:35:48 WARN Utils: Your hostname, Nathans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
22/05/19 11:35:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 11:35:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_path = os.path.join('..', '..', 'src-data', 'flights_small.csv')

# 1. Spark Job

## 1.1. Read CSV without option

In [4]:
# job(0): read
df = spark.read \
    .csv(file_path)

In [5]:
# job 2
df.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
| _c0|  _c1|_c2|     _c3|      _c4|     _c5|      _c6|    _c7|    _c8|   _c9|  _c10|_c11|    _c12|    _c13|_c14|  _c15|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
+----+-----+---+--------+---------+-----

## Result of 1.1

![plot](https://github.com/DatacollectorVN/PySpark-Tutorial/blob/master/public-imgs/ui_1.1.png?raw=true)

## 1.2. Read CSV with option

Note: reset again

In [4]:
# job(0): read
df = spark.read \
    .option('header', True) \
    .option('inferSchema', True) \
    .csv(file_path)
# option('inferSchema') is job(1)

*Note:* option('header') do not affect `Spark Job`.

In [5]:
# job 2
df.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
+----+-----+---+--------+---------+-----

## Result of 1.2

Overview Spark Jobs

![plot](https://github.com/DatacollectorVN/PySpark-Tutorial/blob/master/public-imgs/ui_1.2.png?raw=true)

Detail Job Id 1

![plot](https://github.com/DatacollectorVN/PySpark-Tutorial/blob/master/public-imgs/ui_1.2.detail_stage_1.png?raw=true)

## 1.3. Use method with wide transformation

Narrow and Wide transformation in Spark. Read my own document.

In [4]:
# job(0): read
df = spark.read \
    .option('header', True) \
    .option('inferSchema', True) \
    .csv(file_path)
# option('inferSchema') is job(1)

In [5]:
# job 2
df.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
+----+-----+---+--------+---------+-----

In [6]:
df.where(df.distance > 1000).show(5) # job(3)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|     198|    1721|  18|    12|
|2014|    8|  3|    1120|        0|    1415|        2|     AS| N305AS|   656|   SEA| PHX|     154|    1107|  11|    20|
|2014|   11| 12|    2346|       -4|     217|      -28|     AS| N765AS|   121|   SEA| ANC|     183|    1448|  23|    46|
+----+-----+---+--------+---------+-----

In [7]:
df.groupBy('year').agg({'distance': 'mean'}).show() # job(4) and have skipped stage in job(5)

+----+-------------+
|year|avg(distance)|
+----+-------------+
|2014|    1208.1516|
+----+-------------+



22/05/19 11:44:56 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /private/var/folders/39/ypky8d257z90sg1r9zbcp99c0000gn/T/spark-b3574271-3059-4bd6-8b30-862023f9db98/pyspark-ffcbd4bc-963f-408b-bde2-cd7762d7e44b. Falling back to Java IO way
java.io.IOException: Failed to delete: /private/var/folders/39/ypky8d257z90sg1r9zbcp99c0000gn/T/spark-b3574271-3059-4bd6-8b30-862023f9db98/pyspark-ffcbd4bc-963f-408b-bde2-cd7762d7e44b
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1164)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4$adapted(ShutdownHookManager.scala:62)
	at scala.collection.I

## Result 1.3

![plot](https://github.com/DatacollectorVN/PySpark-Tutorial/blob/master/public-imgs/ui_1.3.png?raw=true)

At detail of Job 5, we see the skipped stage. 

The skipped stage happens when:
- `Cached data`: If the data is cached or persisted by an explicit use of `cache()` or `persist()` you might see a stage being skipped when the result of the stage is already cached.
- `Shuffle data`: `Spark` will automatically cache the data in the stage right after the shuffle. Shuffle is an expensive operation and hence Spark does this automatically. But note, the data will not be available for ever. This data will be evicted using Least Recently Used (LRU) strategy as soon as memory becomes unavailable for newer data.  

In our case, when we use `groupBy()` method --> wide transformation ---> Shuffle data.

![plot](https://github.com/DatacollectorVN/PySpark-Tutorial/blob/master/public-imgs/ui_1.3.detaile_job_5.png?raw=true)